In [1]:
# Car Model Prediction

In [1]:
from src.models import SVM, Classifier
from src.train import train, validate
from src.utils import show_images, load_test_images, load_train_images
from src.reader import ImageFolderWithPaths

import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'\n{device} will be used as device')


cuda:0 will be used as device


In [ ]:
# Load data and display

root = 'data/data'

train_data = load_train_images(root + '/verification/verification_train.txt')
test_data = load_test_images(root + '/verification/verification_pairs_easy.txt')

celeba_train = ImageFolder(root=celeba_root, transform=transforms.Compose([
  transforms.Resize(scale_size),
  transforms.ToTensor(),
]))

batch_size = 128
x_train = None
x_test = None
y_train = None
y_test = None

# mnist = datasets.MNIST('./MNIST_data', train=True, download=True,
#                            transform=transforms.ToTensor())

train_loader = DataLoader(mnist, batch_size=batch_size, drop_last=True)
val_loader = DataLoader(mnist, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(mnist, batch_size=batch_size, drop_last=True)


imgs = data_loader.__iter__().next()[0].numpy().squeeze()
show_images(imgs, color=True)

In [ ]:
#SVM

In [ ]:
learning_rate = 0.001
epochs = 50

model = SVM().to(device)
optim = torch.optim.Adam(model.parameters(), lr=learning_rate, betas = (0.5, 0.999))

# train
svm_losses = train(model, optim, batch_size, num_epochs=epochs, device=device, train_loader=train_loader, val_loader=val_loader)

In [ ]:
# plot error

plt.figure(figsize=(10,5))
plt.title("Loss During Training")
plt.plot(svm_losses)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.show()

In [ ]:
loss, acc = validate(test_loader)
print(f'SVM test accuracy is: {acc}')

In [ ]:
# Classifier

In [ ]:
learning_rate = 0.001
epochs = 50

model = Classifier().to(device)
optim = torch.optim.Adam(model.parameters(), lr=learning_rate, betas = (0.5, 0.999))

# train
classifier_losses = train(model, optim, batch_size, num_epochs=epochs, device=device, train_loader=train_loader, val_loader=val_loader)

In [ ]:
# plot error

plt.figure(figsize=(10,5))
plt.title("Loss During Training")
plt.plot(classifier_losses)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.show()

In [ ]:
loss, acc = validate(test_loader)
print(f'Classifier test accuracy is: {acc}')